In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv(r"C:\Users\USER\UIT-webscraping\articles_paragraphs.csv")

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df = df.rename(columns={'paragraph': 'text'})

In [5]:
df.head(20)

,text,category,source
0,ByChukwuma Umeorah,National,https://thesun.ng/nis-mobilises-customs-police...
1,"TheNigeria Immigration Service (NIS), the Nige...",National,https://thesun.ng/nis-mobilises-customs-police...
2,Speaking at the opening ceremony of the ongoin...,National,https://thesun.ng/nis-mobilises-customs-police...
3,At the workshop themed “Maritime security in t...,National,https://thesun.ng/nis-mobilises-customs-police...
4,"She said: “The theme of this Workshop, Maritim...",National,https://thesun.ng/nis-mobilises-customs-police...
5,"“In addition, the ECOWAS Sub-Committee of Chie...",National,https://thesun.ng/nis-mobilises-customs-police...
6,“This workshop further emphasizes our collecti...,National,https://thesun.ng/nis-mobilises-customs-police...
7,"“Along West Africa’s Atlantic coast, the Gulf ...",National,https://thesun.ng/nis-mobilises-customs-police...
8,“These threats undermine regional stability an...,National,https://thesun.ng/nis-mobilises-customs-police...
9,“As the Comptroller-General of the Nigeria Imm...,National,https://thesun.ng/nis-mobilises-customs-police...


In [6]:
df = df[~df['text'].str.strip().str.match(r'(?i)^(by|from)[a-z]')].reset_index(drop=True)

df['text'] = df['text'].str.replace(r'(?i)^(the)([a-z])', r'\1 \2', regex=True)

In [7]:
df.head(20)

,text,category,source
0,"The Nigeria Immigration Service (NIS), the Nig...",National,https://thesun.ng/nis-mobilises-customs-police...
1,Speaking at the opening ceremony of the ongoin...,National,https://thesun.ng/nis-mobilises-customs-police...
2,At the workshop themed “Maritime security in t...,National,https://thesun.ng/nis-mobilises-customs-police...
3,"She said: “The theme of this Workshop, Maritim...",National,https://thesun.ng/nis-mobilises-customs-police...
4,"“In addition, the ECOWAS Sub-Committee of Chie...",National,https://thesun.ng/nis-mobilises-customs-police...
5,“This workshop further emphasizes our collecti...,National,https://thesun.ng/nis-mobilises-customs-police...
6,"“Along West Africa’s Atlantic coast, the Gulf ...",National,https://thesun.ng/nis-mobilises-customs-police...
7,“These threats undermine regional stability an...,National,https://thesun.ng/nis-mobilises-customs-police...
8,“As the Comptroller-General of the Nigeria Imm...,National,https://thesun.ng/nis-mobilises-customs-police...
9,“Mitigating maritime threats in this vital reg...,National,https://thesun.ng/nis-mobilises-customs-police...


In [8]:
df = df[df['text'].str.split().str.len() >= 5].reset_index(drop=True)

In [9]:
selected_category = df[df['category'] == 'Politics & Power'].reset_index(drop=True)

In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Lowercase, remove punctuation/special characters and extra whitespaces from the text
selected_category['cleaned_text'] = (selected_category['text'].str.lower()
                                     .str.replace(r'[^\w\s]', '', regex=True)
                                     .str.replace(r'\s+', ' ', regex=True).str.strip())

# Remove stop words
filtered_texts = []
for text in selected_category['cleaned_text']:
    words = text.split()               # Split into words
    filtered_words = []                # Prepare empty list for filtered words
    for word in words:
        if word not in stop_words:    # Check if word is NOT a stop word
            filtered_words.append(word)  # If not stop word, add to list
    cleaned_text = ' '.join(filtered_words)  # Join filtered words back into string
    filtered_texts.append(cleaned_text)       # Append cleaned text to list

# Assign the cleaned column back
selected_category['cleaned_text'] = filtered_texts

In [13]:
selected_category.to_csv(r"C:\Users\USER\UIT-webscraping\articles_paragraphs_politics.csv", index=False)